In [33]:
from socceraction.spadl.statsbomb import convert_to_actions
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl.config import actiontypes, bodyparts
import socceraction.spadl as spadl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor
import math
import pickle
from mplsoccer import Pitch
import hashlib
import copy
import random

In [34]:
api = StatsBombLoader(root="data/statsbomb")

In [35]:
# Competitions Input
competitions_df = api.competitions()
competitions_df[competitions_df["country_name"] == "International"]

,season_id,competition_id,competition_name,country_name,competition_gender,season_name
25,274,1470,FIFA U20 World Cup,International,male,1979
26,106,43,FIFA World Cup,International,male,2022
27,3,43,FIFA World Cup,International,male,2018
28,55,43,FIFA World Cup,International,male,1990
29,54,43,FIFA World Cup,International,male,1986
30,51,43,FIFA World Cup,International,male,1974
31,272,43,FIFA World Cup,International,male,1970
32,270,43,FIFA World Cup,International,male,1962
33,269,43,FIFA World Cup,International,male,1958
68,107,72,Women's World Cup,International,female,2023


In [36]:
# Games Input (Competition_id, Season_id)
games_world_cup_2018_df = api.games(43, 3)
games_world_cup_2018_df

,game_id,season_id,competition_id,competition_stage,game_day,game_date,home_team_id,away_team_id,home_score,away_score,venue,referee
0,7585,3,43,Round of 16,4,2018-07-03 20:00:00,769,768,1,1,Otkritie Bank Arena,Mark Geiger
1,7570,3,43,Group Stage,3,2018-06-28 20:00:00,768,782,0,1,Stadion Kaliningrad,Damir Skomina
2,7586,3,43,Round of 16,4,2018-07-03 16:00:00,790,773,1,0,Saint-Petersburg Stadium,Damir Skomina
3,7557,3,43,Group Stage,3,2018-06-25 20:00:00,797,780,1,1,Mordovia Arena,Enrique Cáceres
4,7542,3,43,Group Stage,2,2018-06-20 14:00:00,780,788,1,0,Stadion Luzhniki,Mark Geiger
...,...,...,...,...,...,...,...,...,...,...,...,...
59,7540,3,43,Group Stage,2,2018-06-19 20:00:00,796,774,3,1,Saint-Petersburg Stadium,Enrique Cáceres
60,8652,3,43,Quarter-finals,5,2018-07-07 20:00:00,796,785,2,2,\tOlimpiyskiy Stadion Fisht,Sandro Ricci
61,7563,3,43,Group Stage,3,2018-06-26 16:00:00,776,771,0,0,Stadion Luzhniki,Sandro Ricci
62,7556,3,43,Group Stage,2,2018-06-24 17:00:00,778,787,2,2,\tEkaterinburg Arena,Gianluca Rocchi


In [37]:
# Teams Input (game_id)
teams_world_cup_2018_df = api.teams(7559)
teams_world_cup_2018_df.head()

,team_id,team_name
0,774,Egypt
1,799,Saudi Arabia


In [38]:
# Player Input (game_id)
players_df = api.players(7559)
players_df

,game_id,team_id,player_id,player_name,nickname,jersey_number,is_starter,starting_position_id,starting_position_name,minutes_played
0,7559,774,3531,Mohamed Salah,None,10,True,19,Center Attacking Midfield,101
1,7559,774,3536,Ramadan Sobhi Ramadan Ahmed,Ramadan Sobhi,14,False,0,Substitute,32
2,7559,774,3568,Mohamed Naser Elsayed Elneny,Mohamed Elneny,17,True,13,Right Center Midfield,101
3,7559,774,4063,Ahmed Elsayed Ali Elsayed Hegazy,Ahmed Hegazy,6,True,5,Left Center Back,101
4,7559,774,5250,Ahmed Fathy Abdel Meneim Ibrahim,Ahmed Fathy,7,True,2,Right Back,101
5,7559,774,5251,Abdalla Mahmoud El Said Bekhit,Abdalla El Said,19,True,17,Right Wing,57
6,7559,774,5252,Marwan Mohsen,None,9,True,23,Center Forward,69
7,7559,774,5253,Amr Medhat Warda,Amr Warda,22,False,0,Substitute,44
8,7559,774,5254,Ali Gabr,None,2,True,3,Right Center Back,101
9,7559,774,5257,Mahmoud Abdul Monem Abdelhamid Soliman,Kahraba,11,False,0,Substitute,15


In [39]:
# Events Input (game_id)
event_world_cup_2018_df = api.events(7559)
event_world_cup_2018_df

,game_id,event_id,period_id,team_id,player_id,type_id,type_name,index,timestamp,minute,...,team_name,duration,extra,related_events,player_name,position_id,position_name,location,under_pressure,counterpress
0,7559,46b33c8d-8e83-4d1e-8549-acdb30daf2f0,1,799,NaN,35,Starting XI,1,1900-01-01 00:00:00.000,0,...,Saudi Arabia,0.000,"{'tactics': {'formation': 4141, 'lineup': [{'p...",[],NaN,NaN,NaN,NaN,False,False
1,7559,5a23945a-c063-4fa9-88b9-aae9b330fc7a,1,774,NaN,35,Starting XI,2,1900-01-01 00:00:00.000,0,...,Egypt,0.000,"{'tactics': {'formation': 4231, 'lineup': [{'p...",[],NaN,NaN,NaN,NaN,False,False
2,7559,308ed484-e69c-47a9-8187-ff2cbaab23f4,1,774,NaN,18,Half Start,3,1900-01-01 00:00:00.000,0,...,Egypt,9.053,{},[c70095f4-cbf1-4c44-88be-fb260f405631],NaN,NaN,NaN,NaN,False,False
3,7559,c70095f4-cbf1-4c44-88be-fb260f405631,1,799,NaN,18,Half Start,4,1900-01-01 00:00:00.000,0,...,Saudi Arabia,8.920,{},[308ed484-e69c-47a9-8187-ff2cbaab23f4],NaN,NaN,NaN,NaN,False,False
4,7559,49d14d7d-f30d-4cc9-9782-a961b11cd858,1,774,5252.0,30,Pass,5,1900-01-01 00:00:00.100,0,...,Egypt,0.933,"{'pass': {'recipient': {'id': 3568, 'name': 'M...",[2fd09a99-c322-46e8-a957-4205530ed4c0],Marwan Mohsen,23.0,Center Forward,"[60.0, 40.0]",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,7559,84855116-f3ee-4671-b757-b9ce80d79008,2,799,5187.0,16,Shot,3827,1900-01-01 00:49:05.160,94,...,Saudi Arabia,0.893,"{'shot': {'statsbomb_xg': 0.021029433, 'end_lo...",[2418f991-1467-4599-a115-8b8c8927c5c6],Salem Mohammed Al Dawsari,19.0,Center Attacking Midfield,"[116.0, 53.0]",False,False
3827,7559,2418f991-1467-4599-a115-8b8c8927c5c6,2,774,6292.0,23,Goal Keeper,3828,1900-01-01 00:49:06.053,94,...,Egypt,NaN,"{'goalkeeper': {'position': {'id': 44, 'name':...",[84855116-f3ee-4671-b757-b9ce80d79008],Essam Kamal Tawfik El Hadary,1.0,Goalkeeper,"[1.0, 36.0]",False,False
3828,7559,d0237cd7-d111-411a-a75d-e7f2dfd560aa,2,774,3531.0,30,Pass,3829,1900-01-01 00:50:28.920,95,...,Egypt,0.933,"{'pass': {'recipient': {'id': 5262, 'name': 'T...",[],Mohamed Salah,22.0,Right Center Forward,"[61.0, 41.0]",False,False
3829,7559,02496cdc-24e5-48e6-a9bc-10e598064edc,2,774,NaN,34,Half End,3830,1900-01-01 00:50:33.160,95,...,Egypt,NaN,{},[ea485df9-9d5c-455f-b8ba-48523de17a89],NaN,NaN,NaN,NaN,False,False
